In [256]:
import pandas as pd
import requests

In [243]:
# rankings API request
# don't run this frequently (max 250 per month)


# api call to get the matches for a given day
url = "https://tennis-live-data.p.rapidapi.com/matches-by-date/2022-07-10" # date in yyyy-mm-dd

headers = {
	"X-RapidAPI-Key": "00229723ddmsh7d1fdfd7b967bd6p1e6ca6jsn0b3e25e2c461",
	"X-RapidAPI-Host": "tennis-live-data.p.rapidapi.com"
}

matches_response = requests.request("GET", url, headers=headers)

In [260]:
# unnest returned JSON


# flatten the JSON
matches_df = pd.json_normalize(matches_response.json()['results'])
matches_df = matches_df.explode('matches')
matches_df = matches_df['matches'].apply(pd.Series)[['status', 'home_player', 'away_player', 'date']]

# format names to be joined with ranking data
matches_df['home_player'] = matches_df['home_player'].apply(lambda x: str(x.split()[0]))
matches_df['away_player'] = matches_df['away_player'].apply(lambda x: str(x.split()[0]))
matches_df.head()

# duplicate rows to make the dataframe tidy by player
opp = matches_df.rename(columns={
    'home_player': 'away_player',
    'away_player': 'home_player'
})

matches_df = pd.concat([matches_df, opp]).rename(columns={
    'home_player': 'player',
    'away_player': 'opponent'
})

matches_df.head()

,status,player,opponent,date
0,finished,Djokovic,Kyrgios,2022-07-10T13:00:00+00:00
1,finished,Flink,Papamichail,2022-07-10T09:00:00+00:00
1,finished,Baindl,Stolmar,2022-07-10T09:15:00+00:00
1,finished,Talaba,Alves,2022-07-10T11:20:00+00:00
1,finished,Pera,Shapatava,2022-07-10T11:10:00+00:00


In [245]:
# rankings API request
# don't run this frequently (max 250 per month)


# api call to get the rank points for ATP players
url = "https://tennis-live-data.p.rapidapi.com/rankings/ATP"

headers = {
	"X-RapidAPI-Key": "00229723ddmsh7d1fdfd7b967bd6p1e6ca6jsn0b3e25e2c461",
	"X-RapidAPI-Host": "tennis-live-data.p.rapidapi.com"
}

rankings_response = requests.request("GET", url, headers=headers)

In [265]:
# unnest returned JSON


# flatten the JSON and rename columns
rankings_df = pd.json_normalize(response.json()['results']['rankings'])
rankings_df = rankings_df[['full_name', 'ranking_points']]
rankings_df['player'] = rankings_df['full_name'].apply(lambda x: str(x.split()[-1]))
rankings_df = rankings_df.rename(columns={'full_name': 'player_full_name', 'ranking_points': 'player_ranking_points'})
rankings_df.head()

,player_full_name,player_ranking_points,player
0,Daniil Medvedev,7955,Medvedev
1,Alexander Zverev,7030,Zverev
2,Novak Djokovic,6770,Djokovic
3,Rafael Nadal,6525,Nadal
4,Stefanos Tsitsipas,5150,Tsitsipas


In [269]:
# joining the columns (could be cleaned up)


# merge to get player names and rank points
matches_df['player']=matches_df['player'].astype(str)
rankings_df['player']=rankings_df['player'].astype(str)
info_df = matches_df.merge(rankings_df, on='player', how='left')

# merge to get opponent names and rank points
opp_rankings_df = rankings_df.rename(columns={'player_full_name': 'opp_full_name', 'player_ranking_points': 'opp_ranking_points', 'player': 'opponent'})
info_df = info_df.merge(opp_rankings_df, on='opponent', how='left')

# return just ATP matches
info_df = info_df[pd.notnull(info_df['player_full_name'])]
matches = info_df.drop(columns = info_df[['player', 'opponent']])

matches.head()

,status,date,player_full_name,player_ranking_points,opp_full_name,opp_ranking_points
0,finished,2022-07-10T13:00:00+00:00,Novak Djokovic,6770.0,Nick Kyrgios,1070.0
6,finished,2022-07-10T13:00:00+00:00,Nick Kyrgios,1070.0,Novak Djokovic,6770.0
